## How to create a custom executor

Executors define the low-level directions for the computation. They can specify different capabilities, eg., different hardware, different computation strategy, different logic, or simply different goals.

Executors are plugins; any executor-plugins which are found are imported as classes in the covalent.executor name-space.
See the how-to on [choosing an executor to be used in an electron](choosing_executors.ipynb) for details on simply choosing an executor.
    
You can write your own executor to execute Covalent electrons in any way you like, using particular environments, cloud resources, or hardware.

A template to write one can be found [here](https://github.com/AgnostiqHQ/covalent-executor-template).

Following things should be part of the plugin file:

1. `_EXECUTOR_PLUGIN_DEFAULTS`, any defaults for the executor
2. `executor_plugin_name`, name of the executor to make it importable by `covalent.executors` - must be the same as its class name
3. `run()`, core logic of how the executor will handle the user function, its args and its kwargs

As an example, a `DockerExecutor` - to execute electrons in Docker containers, is constructed in this how-to.

The command to be executed in the container can be rendered as a multiline string. To this end, we create a method `_format_exec_command` in our `DockerExecutor` to facilitate generation of the `exec` command, and override the `run()` method to utilize it:

In [ ]:
import os
import copy
import shutil
import cloudpickle as pickle
import tempfile
import subprocess
from typing import List, Dict, Callable

from covalent.executor import BaseExecutor
from covalent._shared_files import logger

_EXECUTOR_PLUGIN_DEFAULTS = {
    "image": "python:3.8",   # default docker image to be used if not specified
    "workdir": ".", # default host workdir
    "options": {},    # default options supported by `docker run`
}

executor_plugin_name = 'DockerExecutor'

class DockerExecutor(BaseExecutor):

    def __init__(self,
        image: str, # name of the docker image to use
        workdir: str, # host workdir
        options: Dict = None, # dictionary of options supported by `docker run`
        container_workdir: str = "/tmp/covalent",  # workdir inside the container
        *args,
        **kwargs
    ):
        self.image = image
        self.wordir = workdir
        self.options = options or {}
        self._container_workdir = container_workdir

        super().__init__(*args, **kwargs)

    def _format_exec_command(self, func_filename: str, result_filename: str) -> str:

        return "\n".join(
            [
                "import cloudpickle as pickle",
                "",
                "result = None",
                "exception = None",
                "",
                "with open('{func_filename}', 'rb') as f:",
                "   function, args, kwargs = pickle.load(f)",
                "   try:",
                "       result = function(*args, **kwargs)",
                "   except Exception as e:",
                "       exception = e",
                "   finally:",
                "       with open('{result_filename}', 'wb') as f_out:",
                "           pickle.dump((result, exception), f_out)",
                "",
            ]
        ).format(
            func_filename = os.path.join(self._container_workdir, func_filename),
            result_filename = os.path.join(self._container_workdir, result_filename)
        )


    def run(self,
        function: Callable, # Input python function
        args: List, # list of positional args
        kwargs: Dict, # dictionary of keyword args
        task_metadata: Dict, # dictionary of metadata, contains "dispatch_id" and "node_id"
    ):

        result_filename = f"result-{task_metadata['dispatch_id']}-{task_metadata['node_id']}.pkl"
        container_name = f"container-{task_metadata['dispatch_id']}-{task_metadata['node_id']}"
        execute_script_name = f"dkrexec-{task_metadata['dispatch_id']}-{task_metadata['node_id']}.py"

        with tempfile.NamedTemporaryFile(dir=self.workdir) as f, tempfile.NamedTemporaryFile(dir=self.workdir, mode="w") as g:

                pickle.dump((function, args, kwargs), f)
                f.flush()

                # Format the command to be executed inside the container
                func_filename = f"func-{task_metadata['dispatch_id']}-{task_metadata['node_id']}.pkl"
                shutil.copy(f.name, os.path.join(self.workdir, func_filename))

                cmd = self.format_exec_command(func_filename, result_filename)
                g.write(cmd)
                g.flush()
                shutil.copy(g.name, os.path.join(self.workdir, execute_script_name))

                # Start the container in detached mode
                subprocess.run(
                    [
                        "docker",
                        "container",
                        "run",
                        "-dit",
                        "--rm",
                        "--mount",
                        f"type=bind,source={self.workdir},target={self._container_workdir}",
                        "--name",
                        container_name,
                        self.image
                    ],
                    check=True,
                    capture_output=True,
                )

                # Execute the script/command inside the container
                subprocess.run(
                    [
                        "docker",
                        "container",
                        "exec",
                        container_name,
                        "python",
                        f"{self._container_workdir}/{execute_script_name}"
                    ],
                    check=True,
                    capture_output=True
                )

                # Assert that a result object was created
                assert os.path.exists(os.path.join(self.workdir, result_filename))

                # Read the generated result object from the result pickle file
                with open(os.path.join(self.workdir, result_filename), "rb") as f_read:
                    result, exception = pickle.load(f_read)
                    if exception:
                        raise exception

                # Terminate the container
                subprocess.run(
                    [
                        "docker",
                        "container",
                        "stop",
                        container_name
                    ],
                    check=True,
                    capture_output=True
                )

                # Return
                return result


Assuming an image named `docker-executor-demo:latest` has already been built with Covalent installed, this executor can now be used as part of workflows to execute electons inside the container:

In [ ]:
import os
import covalent as ct

docker = DockerExecutor(image='docker-executor-demo:latest', workdir=os.getcwd())

@ct.electron(executor=docker)
def add(x, y):
    return x + y

@ct.electron(executor=docker)
def multiply(x, y):
    return x*y


@ct.lattice
def workflow(x, y):
    r1 = add(x, y)
    return multiply(y, r1)

# result = ct.dispatch_sync(workflow)(2, 3)
# print(result)

```bash
Lattice Result
==============
status: COMPLETED
result: 15
inputs: {'args': [2, 3], 'kwargs': {}}
error: None

start_time: 2022-07-18 16:41:48.106224+00:00
end_time: 2022-07-18 16:41:55.053115+00:00

results_dir: results
dispatch_id: 865fb0fd-7be3-4d15-a24d-97e5327b4253

Node Outputs
------------
add(0): 5
:parameter:2(1): 2
:parameter:3(2): 3
multiply(3): 15
:parameter:3(4): 3
```